In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm 

tqdm.pandas()

In [2]:
df = pd.read_csv("q_quora.csv")
df.sample(5)

C:\Users\HP\AppData\Local\Temp\ipykernel_17356\3667460292.py:1: DtypeWarning: Columns (7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("q_quora.csv")


,id,qid1,qid2,question1,question2,is_duplicate,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
96617,96617,191778,191779,"When world going to end, what you'll do at the...",The Future: How will the world end?,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211827,211827,417780,417781,What are your favorite movies of all time? It ...,What are the must watch movies to see before y...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
328455,328455,644173,644174,What is some good free 2D animation software (...,What is the best 2d animation software?,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
286251,286251,562555,562556,I got 101 marks in mh cet law and I have been ...,What is the best early morning routine for stu...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346573,346573,679129,679130,Is it possible to believe in the bible (Christ...,Is it possible to account for the Hebrew Bible...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df.iloc[:, 3:6]

<h1>Start with text Pre-processing</h1>

In [4]:
df.sample(5)

,question1,question2,is_duplicate
216094,Does the black hole hold a gateway to another ...,Would be the black hole another universe simil...,1
342859,How would you know your age?,How can we know age of a fish?,0
86158,What is Hillary Clinton 's Foreign policy rega...,"I have no facial hair, and I am 24 with 6'6 he...",0
395863,Like comments on Instagram?,How do I tell a girl I like her (like none of ...,0
333148,What does it say about America that a misogyni...,What would be a good way to word a sign for my...,0


In [5]:
def make_int(num):
    try:
        return int(num)
    except:
        return -1
        
df['is_duplicate'] = df['is_duplicate'].apply(make_int)

In [6]:
df['is_duplicate'].value_counts()

is_duplicate
 0    254748
 1    149267
-1       336
Name: count, dtype: int64

In [7]:
df = df.drop(df[df['is_duplicate'] == -1].index)
df['is_duplicate'].value_counts()

is_duplicate
0    254748
1    149267
Name: count, dtype: int64

In [8]:
df.shape

(404015, 3)

In [9]:
df.sample(5)

,question1,question2,is_duplicate
120786,How do Germans learn German?,Why might one learn German?,0
232630,Does anybody know what 'nanpa/nampa' means in ...,"What is your review of Osaka, Japan?",0
60493,Why haven't there been any further moon landin...,Why did NASA discontinue the Moon landings?,1
206896,What do I feed my 5-month-old Rottweiler?,What do I feed my 6 month old Rottweiler?,0
196888,How do I increase my learning speed?,How can I increase my learning speed?,1


In [10]:
df.drop(df[df['question1'].isna()].index, inplace = True)
df.drop(df[df['question2'].isna()].index, inplace = True)
df['question2'].isna().sum()

0

In [11]:
X = df.iloc[:, :2]
y = df.iloc[:, 2]
X.head(5)

,question1,question2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


<h2>Lower Casing</h2>

In [12]:
X['question1'] = X['question1'].apply(lambda x : x.lower())
X['question2'] = X['question2'].apply(lambda x : x.lower())

df.head(5)

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


<h2>Remove HTML tags and URLs</h2>
<h4>Since there are no HTML tags and URLs we can move forward</h4>

<h2>Remove Punctuation marks</h2>

In [13]:
import string

punctuation_marks = string.punctuation

# We excluded '!' & '@' from punctuations as it could give useful information. 
punctuation_marks = punctuation_marks[1:20] + punctuation_marks[21:]

def remove_punctuations(text):
    return text.translate(str.maketrans(punctuation_marks, ' '*len(punctuation_marks)))

X['question1'] = X['question1'].apply(remove_punctuations)
X['question2'] = X['question2'].apply(remove_punctuations)
X.head(10)

,question1,question2
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...
2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,why am i mentally very lonely? how can i solve...,find the remainder when math 23 24 math i...
4,which one dissolve in water quikly sugar salt...,which fish would survive in salt water?
5,astrology i am a capricorn sun cap moon and c...,i m a triple capricorn sun moon and ascendan...
6,should i buy tiago?,what keeps childern active and far from phone ...
7,how can i be a good geologist?,what should i do to be a great geologist?
8,when do you use シ instead of し?,when do you use instead of and ?
9,motorola company can i hack my charter moto...,how do i hack motorola dcx3400 for free internet?


<h2>Chat Word Removal</h2>

<h4>Not much required as didn't find any</h4>

In [14]:
# import csv
# import re

# from tqdm import tqdm 

# tqdm.pandas()

# # Git hub link for the same can be found here : https://github.com/rishabhverma17/sms_slang_translator
# def chat_word_converter(user_string):
#     user_string = user_string.split(" ")
#     j = 0
#     for _str in user_string:
#         # File path which consists of Abbreviations.
#         fileName = "slang.txt"
#         # File Access mode [Read Mode]
#         accessMode = "r"
#         with open(fileName, accessMode) as myCSVfile:
#             # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
#             dataFromFile = csv.reader(myCSVfile, delimiter="=")
#             # Removing Special Characters.
#             _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
#             for row in dataFromFile:
#                 # Check if selected word matches short forms[LHS] in text file.
#                 if _str.upper() == row[0]:
#                     # If match found replace it with its Abbreviation in text file.
#                     user_string[j] = row[1]
#             myCSVfile.close()
#         j = j + 1
#     # Replacing commas with spaces for final output.
#     return ' '.join(user_string).lower()

# # print(chat_word_converter("Hi bro ltns"))

# X['question1'] = X['question1'].progress_apply(chat_word_converter)
# X['question2'] = X['question2'].progress_apply(chat_word_converter)
# X.sample(5)

<h2>Spelling checking</h2>

In [15]:
# from spello.model import SpellCorrectionModel
# import pickle
# sp = SpellCorrectionModel(language = "en")
# sp.load("en.pkl")

In [16]:
# def spell_correction(text):
#     p = sp.spell_correct(text)
#     return p['spell_corrected_text']

# X['question1'] = X['question1'].progress_apply(spell_correction)
# X['question2'] = X['question2'].progress_apply(spell_correction)

In [17]:
X.head(10)

,question1,question2
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...
2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,why am i mentally very lonely? how can i solve...,find the remainder when math 23 24 math i...
4,which one dissolve in water quikly sugar salt...,which fish would survive in salt water?
5,astrology i am a capricorn sun cap moon and c...,i m a triple capricorn sun moon and ascendan...
6,should i buy tiago?,what keeps childern active and far from phone ...
7,how can i be a good geologist?,what should i do to be a great geologist?
8,when do you use シ instead of し?,when do you use instead of and ?
9,motorola company can i hack my charter moto...,how do i hack motorola dcx3400 for free internet?


<h2>Remove Stop Words</h2>

In [18]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
en = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
# Except not we take all the stop words as not will play an in important role in determining sentiment of the questions

new_en = []
for word in en:
    if word not in  ["not", "isn't"] :
        new_en.append(word)

en = new_en

In [20]:
en

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [21]:
def remove_stopwords(text):
    new_text = []
    words = text.split(" ")
    for word in words:
        if word not in en:
            new_text.append(word)
    new_text = new_text[:]      
    return " ".join(new_text)     

In [22]:
X['question1'] = X['question1'].progress_apply(remove_stopwords)
X['question2'] = X['question2'].progress_apply(remove_stopwords)

100%|████████████████████████████████████████████████████████████████████████| 404012/404012 [00:06<00:00, 63201.94it/s]


In [23]:
X.head(10)

,question1,question2
0,step step guide invest share market india?,step step guide invest share market?
1,story kohinoor koh noor diamond?,would happen indian government stole kohinoor ...
2,increase speed internet connection using vpn?,internet speed increased hacking dns?
3,mentally lonely? solve it?,find remainder math 23 24 math divided 24...
4,one dissolve water quikly sugar salt methane...,fish would survive salt water?
5,astrology capricorn sun cap moon cap rising ...,triple capricorn sun moon ascendant capricor...
6,buy tiago?,keeps childern active far phone video games?
7,good geologist?,great geologist?
8,use シ instead し?,use instead ?
9,motorola company hack charter motorolla dcx...,hack motorola dcx3400 free internet?


<h2>Handeling Emojis</h2>

<h4>There are no emojis so not needed</h4>

<h2>Tokenization</h2>


In [24]:
# import spacy

# nlp = spacy.load("en_core_web_sm")

# def tokenizer(text):
#     tokens = nlp(text)
#     rtn = []
#     for token in tokens:
#         rtn.append(str(token))
#     return rtn

In [25]:
# X['question1'] = X['question1'].progress_apply(tokenizer)
# X['question2'] = X['question2'].progress_apply(tokenizer)

In [26]:
X.head(10)

,question1,question2
0,step step guide invest share market india?,step step guide invest share market?
1,story kohinoor koh noor diamond?,would happen indian government stole kohinoor ...
2,increase speed internet connection using vpn?,internet speed increased hacking dns?
3,mentally lonely? solve it?,find remainder math 23 24 math divided 24...
4,one dissolve water quikly sugar salt methane...,fish would survive salt water?
5,astrology capricorn sun cap moon cap rising ...,triple capricorn sun moon ascendant capricor...
6,buy tiago?,keeps childern active far phone video games?
7,good geologist?,great geologist?
8,use シ instead し?,use instead ?
9,motorola company hack charter motorolla dcx...,hack motorola dcx3400 free internet?


<h4>I tried using spacy but it was taking a lot of time, first let's do it with split method. First '?' and '!' needs to be seperated. </h4>

In [27]:
import re

def tokenizer(text):
    text = re.findall(r'[^?]+|[^!]+|\!|\?', text)
    lst = []
    for txt in text:
        txt = re.split(r'\s+', txt)
        for w in txt:
            lst.append(w)
    return lst

In [28]:
X['question1'] = X['question1'].progress_apply(tokenizer)
X['question2'] = X['question2'].progress_apply(tokenizer)

100%|████████████████████████████████████████████████████████████████████████| 404012/404012 [00:08<00:00, 50075.69it/s]


In [29]:
X.head(10)

,question1,question2
0,"[step, step, guide, invest, share, market, ind...","[step, step, guide, invest, share, market, ?]"
1,"[story, kohinoor, koh, noor, diamond, ?]","[would, happen, indian, government, stole, koh..."
2,"[increase, speed, internet, connection, using,...","[internet, speed, increased, hacking, dns, ?]"
3,"[mentally, lonely, ?, solve, it?]","[find, remainder, math, 23, 24, math, divided,..."
4,"[one, dissolve, water, quikly, sugar, salt, me...","[fish, would, survive, salt, water, ?]"
5,"[astrology, capricorn, sun, cap, moon, cap, ri...","[triple, capricorn, sun, moon, ascendant, capr..."
6,"[buy, tiago, ?]","[keeps, childern, active, far, phone, video, g..."
7,"[good, geologist, ?]","[great, geologist, ?]"
8,"[use, シ, instead, し, ?]","[use, instead, , ?]"
9,"[motorola, company, hack, charter, motorolla, ...","[hack, motorola, dcx3400, free, internet, ?]"


<h2>Stemming</h2>

In [30]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()

def stemmer(words):
    rtn = []
    for word in words:
        rtn.append(ps.stem(word))

    return rtn

In [31]:
X['question1'] = X['question1'].progress_apply(stemmer)
X['question2'] = X['question2'].progress_apply(stemmer)

100%|████████████████████████████████████████████████████████████████████████| 404012/404012 [00:36<00:00, 11104.06it/s]


In [32]:
X.head(10)

,question1,question2
0,"[step, step, guid, invest, share, market, indi...","[step, step, guid, invest, share, market, ?]"
1,"[stori, kohinoor, koh, noor, diamond, ?]","[would, happen, indian, govern, stole, kohinoo..."
2,"[increas, speed, internet, connect, use, vpn, ?]","[internet, speed, increas, hack, dn, ?]"
3,"[mental, lone, ?, solv, it?]","[find, remaind, math, 23, 24, math, divid, 24,..."
4,"[one, dissolv, water, quikli, sugar, salt, met...","[fish, would, surviv, salt, water, ?]"
5,"[astrolog, capricorn, sun, cap, moon, cap, ris...","[tripl, capricorn, sun, moon, ascend, capricor..."
6,"[buy, tiago, ?]","[keep, childern, activ, far, phone, video, gam..."
7,"[good, geologist, ?]","[great, geologist, ?]"
8,"[use, シ, instead, し, ?]","[use, instead, , ?]"
9,"[motorola, compani, hack, charter, motorolla, ...","[hack, motorola, dcx3400, free, internet, ?]"


<h1>Now Since we have done Pre-Preocessing, Now word vectorisation</h1>

<h2>We can use BagOfWords</h2>

In [312]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [334]:
# import and instantiate the vectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(lowercase = False, ngram_range = (1, 1), max_features = 20000)

def join_tokens(txt):
    return " ".join(txt) 

X_train_temp = {}

X_train_temp['question1'] = cv.fit_transform(X_train['question1'].progress_apply(join_tokens))
X_train_temp['question2'] = cv.transform(X_train['question2'].progress_apply(join_tokens))
# X_test_temp = cv.transform(X_test)

100%|███████████████████████████████████████████████████████████████████████| 323209/323209 [00:00<00:00, 571511.78it/s]


In [324]:
cv.get_feature_names_out().shape

(10000,)

In [335]:
X_test_temp = {}

X_test_temp['question1'] = cv.transform(X_test['question1'].progress_apply(join_tokens))
X_test_temp['question2'] = cv.transform(X_test['question2'].progress_apply(join_tokens))

100%|█████████████████████████████████████████████████████████████████████████| 80803/80803 [00:00<00:00, 557128.40it/s]


In [326]:
X_train_temp

{'question1': <323209x10000 sparse matrix of type '<class 'numpy.int64'>'
 	with 1661288 stored elements in Compressed Sparse Row format>,
 'question2': <323209x10000 sparse matrix of type '<class 'numpy.int64'>'
 	with 1683394 stored elements in Compressed Sparse Row format>}

In [336]:
X_train_temp['question1'] = X_train_temp['question1'][:10000].toarray()
X_train_temp['question2'] = X_train_temp['question2'][:10000].toarray()
y_train = y_train[:10000]

X_train_temp['question1'] = X_train_temp['question1'][:10000].tolist()
X_train_temp['question2'] = X_train_temp['question2'][:10000].tolist()
y_train = y_train[:10000]

In [337]:
X_test_temp['question1'] = X_test_temp['question1'][:1000].toarray().tolist()
X_test_temp['question2'] = X_test_temp['question2'][:1000].toarray().tolist()

In [338]:
y_test = y_test[:1000]

In [339]:
X_test_temp = pd.DataFrame(X_test_temp)
X_train_temp = pd.DataFrame(X_train_temp)

In [218]:
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.preprocessing import FunctionTransformer


# class CosineSimilarityTransformer(BaseEstimator) :
#     def __init__(self):
#         self.count = 0
#         pass
    
#     def cosine_sim(self, X):
#         vector1 = np.array(X[0]).reshape(-1, 1)
#         vector2 = np.array(X[1]).reshape(-1, 1)
#             # print(vector1.shape)
#         try:
#             mult = (vector1.T@vector2)/(np.linalg.norm(vector1, 2)*np.linalg.norm(vector2, 2))
#             return mult
#         except:
#             return 0
        
#     def fit(self, X, y=None):
#         self.count = self.count + 1
#         print(self.count)
#         lst = []
#         for x in X:
#             lst.append(self.cosine_sim(x))

#         ans = {}
#         ans['X'] = lst
#         ans['y'] = y
#         return pd.DataFrame(ans)
        
#     def transform(self, X):
#         lst = []
#         for x in X:
#             lst.append(self.cosine_sim(x))

#         ans = {}
#         ans['X'] = lst
#         return pd.DataFrame(ans)        
#     def predict(self, X):
#         return self.cosine_sim(X[0], X[1])

In [219]:
# cosine_sim_pipeline = Pipeline([
#     ('cosine_similarity', CosineSimilarityTransformer()),
#     ('logistic_regression', LogisticRegression())
# ])

In [220]:
# questions_train = list(zip(X_train_temp['question1'], X_train_temp['question2']))
# questions_test = list(zip(X_test_temp['question1'], X_test_temp['question2']))

In [221]:
# np.array(questions_train).shape

In [222]:
# cosine_sim_pipeline.fit(questions_train, y_train)

In [259]:
def cosine_sim(X):
    vector1 = np.array(X[0]).reshape(-1, 1)
    vector2 = np.array(X[1]).reshape(-1, 1)
    try:
        if np.linalg.norm(vector1, 2) == 0 or np.linalg.norm(vector2, 2) == 0:
            return [[0]]
        mult = (vector1.T@vector2)/(np.linalg.norm(vector1, 2)*np.linalg.norm(vector2, 2))
        return mult
    except:
        return 0

print(cosine_sim(questions_train[0])[0][0])

0.7071067811865475


In [213]:
def compute_cosine_sim(row):
    return cosine_sim([row['question1'], row['question2']])[0][0]

In [340]:
X_train_temp['cosine_similarity'] = X_train_temp.progress_apply(compute_cosine_sim, axis = 1)
X_test_temp['cosine_similarity'] = X_test_temp.progress_apply(compute_cosine_sim, axis = 1)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 232.33it/s]


In [320]:
X_train_temp.isna().sum()

question1            0
question2            0
cosine_similarity    0
dtype: int64

In [341]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

X_train_cosine = pd.DataFrame(X_train_temp['cosine_similarity'])
X_test_cosine = pd.DataFrame(X_test_temp['cosine_similarity'])

model.fit(X_train_cosine, y_train)

LogisticRegression()

In [342]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test_cosine)

accuracy_score(y_test, y_pred)

0.637